In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sexo: 1 para mujer y 2 para hombre.


edad: del paciente.


clasificación: resultados de la prueba covid. Los valores 1-3 significan que el paciente fue diagnosticado con covid en diferentes
grados. 4 o superior significa que el paciente no es portador de covid o que la prueba no es concluyente.


tipo de paciente: tipo de atención que recibió el paciente en la unidad. 1 por regreso a casa y 2 por hospitalización.


neumonía: si el paciente ya tiene inflamación de los alvéolos o no.


Embarazo: si la paciente está embarazada o no.


diabetes: si el paciente tiene diabetes o no.


EPOC: Indica si el paciente tiene enfermedad pulmonar obstructiva crónica o no.


asma: si el paciente tiene asma o no.


inmsupr: si el paciente está inmunodeprimido o no.


hipertensión: si el paciente tiene hipertensión o no.


cardiovascular: si el paciente tiene una enfermedad relacionada con el corazón o los vasos sanguíneos.


crónico renal: si el paciente tiene enfermedad renal crónica o no.


otra enfermedad: si el paciente tiene otra enfermedad o no.


Obesidad: si el paciente es obeso o no.


tabaco: si el paciente es consumidor de tabaco.


usmr: Indica si el paciente atendió unidades médicas de primer, segundo o tercer nivel.


unidad médica: tipo de institución del Sistema Nacional de Salud que brinda la atención.


intubado: si el paciente estaba conectado al ventilador.


icu: Indica si el paciente estuvo ingresado en una Unidad de Cuidados Intensivos.


fecha de fallecimiento: Si el paciente falleció indicar la fecha de fallecimiento, y 9999-99-99 en caso contrario.

In [2]:
df = pd.read_csv('covid.csv')
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [3]:
df.describe()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
count,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000,97283.000000
mean,1.511076,3.794764,1.553910,1.529476,47.481328,2.416918,50.824399,54.657525,1.919318,2.120288,2.124749,2.129313,1.842758,2.109454,2.117883,1.877183,2.100562,2.090345,3.844382,47.643545
std,0.499880,0.419409,0.497088,0.499133,47.647595,8.577624,17.532792,47.224271,3.811581,3.809287,3.821103,3.845328,3.573204,4.006936,3.834721,2.553642,3.787255,4.021714,1.593340,47.501829
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,4.000000,1.000000,1.000000,2.000000,1.000000,38.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000
50%,2.000000,4.000000,2.000000,2.000000,2.000000,2.000000,50.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000
75%,2.000000,4.000000,2.000000,2.000000,97.000000,2.000000,64.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,97.000000
max,2.000000,4.000000,2.000000,2.000000,99.000000,99.000000,107.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,7.000000,99.000000


In [4]:
df = df.drop(['DATE_DIED'], axis='columns')

In [5]:
df.PATIENT_TYPE.unique()

array([1, 2], dtype=int64)

In [6]:
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,97,1,65,2,2,2,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,97,1,72,97,2,2,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,1,2,55,97,1,2,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,97,2,53,2,2,2,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,97,2,68,97,1,2,2,2,1,2,2,2,2,2,3,97


In [7]:
# Data X seran las columnas de AGE, HIPERTENSION, CARDIOVASCULAR y TOBACCO
# Data Y PATIENT_TYPE (si se Hospitalizo (2) o retorno a casa(1))

In [8]:
data_x = df.drop(['USMER','MEDICAL_UNIT','SEX','PATIENT_TYPE','INTUBED','PNEUMONIA','PREGNANT','DIABETES','COPD',	
                'ASTHMA','INMSUPR','OTHER_DISEASE','OBESITY','RENAL_CHRONIC','CLASIFFICATION_FINAL','ICU'],axis='columns')
data_y = df['PATIENT_TYPE']

In [9]:
data_x.head()

,AGE,HIPERTENSION,CARDIOVASCULAR,TOBACCO
0,65,1,2,2
1,72,1,2,2
2,55,2,2,2
3,53,2,2,2
4,68,1,2,2


In [10]:
# scal = StandardScaler()
# data_x= scal.fit_transform(data_x)

In [11]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.1)

In [12]:
# Tensores
t_x_train = torch.from_numpy(train_x.values).to("cpu")
t_x_test = torch.from_numpy(test_x.values).to("cpu")
t_y_train = torch.from_numpy(train_y.values).to("cpu")
t_y_test = torch.from_numpy(test_y.values).to("cpu")
t_y_train = t_y_train[:,None]
t_y_test = t_y_test[:,None]

In [13]:
class Red(nn.Module):

    def __init__(self, nro_inputs):
        super(Red,self).__init__()
        self.layer1 = nn.Linear(nro_inputs,12)
        self.layer2 = nn.Linear(12, 20)
        self.layer3 = nn.Linear(20,16)
        self.layer4 = nn.Linear(16,1)

    def forward(self, inputs):
        inputs = inputs.float()
        predict1 = torch.sigmoid(input=self.layer1(inputs))
        predict2 = torch.sigmoid(input=self.layer2(predict1))
        predict3 = torch.sigmoid(input=self.layer3(predict2))
        predict_final = torch.sigmoid(input=self.layer4(predict3))
        return predict_final

In [16]:


lr = 0.001
epochs =1000
status_report = 200

model = Red(4)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam( params = model.parameters(), lr=lr)

print(f'arqui {model}')

for epoch in range(1, epochs+1):
    y_pred = model(t_x_train)
    loss = loss_fn(input=y_pred, target = t_y_train.float())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % status_report == 0:
        print(f'Epoca Nro:  {epoch} - Perdida: {loss.item()}')
    with torch.no_grad():
        y_pred = model(t_x_test)
        y_pred_class = y_pred.round()
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % status_report == 0:
            print(f'Epoca Nro:  {epoch} - Accuracy: {accuracy.item()}')
print(f'Accuracy final: {round(accuracy.item(),4)}')    

arqui Red(
  (layer1): Linear(in_features=4, out_features=12, bias=True)
  (layer2): Linear(in_features=12, out_features=20, bias=True)
  (layer3): Linear(in_features=20, out_features=16, bias=True)
  (layer4): Linear(in_features=16, out_features=1, bias=True)
)
Epoca Nro:  200 - Perdida: -1.0310715436935425
Epoca Nro:  200 - Accuracy: 47.22993087768555
Epoca Nro:  400 - Perdida: -2.3131091594696045
Epoca Nro:  400 - Accuracy: 47.22993087768555
Epoca Nro:  600 - Perdida: -3.745870351791382
Epoca Nro:  600 - Accuracy: 47.22993087768555
Epoca Nro:  800 - Perdida: -5.141951560974121
Epoca Nro:  800 - Accuracy: 47.22993087768555
Epoca Nro:  1000 - Perdida: -6.698533058166504
Epoca Nro:  1000 - Accuracy: 47.22993087768555
Accuracy final: 47.2299


In [24]:
age = 29
hipertension = 1
cardiovascular = 2
tobacco = 1
label_y = {1:'No hospitalizado, retorno a casa', 2:'Hospitalizado'}
result = model(torch.tensor([age,hipertension,cardiovascular,tobacco]))
print(f'Probabilidad de hospitalizacion: {label_y[result.round().item()]}')

Probabilidad de hospitalizacion: No hospitalizado, retorno a casa
